# import

In [25]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [26]:
ratings = pd.read_csv('data/ratings.csv')
ratings
##too big!

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


In [27]:
ratings.user_id.nunique()

53424

In [28]:
def sparsity(df):
    return len(df) / (df.user_id.nunique() * df.book_id.nunique())

sparsity(ratings)

0.011186880428271938

In [29]:
from sklearn.model_selection import train_test_split
_, random_users = train_test_split(ratings.user_id.unique(), test_size=0.05, random_state=322)

ratings_random = ratings.loc[ratings['user_id'].isin(random_users)]
ratings_random

,user_id,book_id,rating
629,58,26,1
630,58,58,4
631,58,18,5
632,58,14,3
633,58,2,3
...,...,...,...
5976331,17635,1922,5
5976455,10622,6810,4
5976457,10622,610,5
5976458,53358,253,5


In [30]:
sparsity(ratings_random)

0.011170451889014762

In [31]:
ratings_random.to_csv('data/ratings_random.csv', index=False)

# visual

In [32]:
import matplotlib.pyplot as plt

In [33]:
#plt.bar('id', 'ratings_count', data=books.sort_values('ratings_count', ascending=False))

In [34]:
books = pd.read_csv('data/books.csv')

In [35]:
books.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [36]:
books = books[['id', 'book_id', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'title', 'language_code']]
books.head()

,id,book_id,isbn,isbn13,authors,original_publication_year,title,language_code
0,1,2767052,439023483,9.780439e+12,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)",eng
1,2,3,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,eng
2,3,41865,316015849,9.780316e+12,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)",en-US
3,4,2657,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,eng
4,5,4671,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng


In [37]:
books.language_code.unique()

array(['eng', 'en-US', 'en-CA', nan, 'spa', 'en-GB', 'fre', 'nl', 'ara',
       'por', 'ger', 'nor', 'jpn', 'en', 'vie', 'ind', 'pol', 'tur',
       'dan', 'fil', 'ita', 'per', 'swe', 'rum', 'mul', 'rus'],
      dtype=object)

In [38]:
books.fillna('unknown').groupby('language_code').aggregate({'id':'count'}).sort_values('id', ascending=False).transpose()

language_code,eng,en-US,unknown,en-GB,ara,en-CA,fre,ind,spa,ger,...,nor,fil,ita,tur,swe,mul,rus,rum,nl,vie
id,6341,2070,1084,257,64,58,25,21,20,13,...,3,2,2,1,1,1,1,1,1,1


In [39]:
pd.merge(ratings, books[['id', 'language_code']], left_on='book_id', right_on='id', how='left').fillna('unknown') \
.groupby('language_code').aggregate({'rating':'count'}).sort_values('rating', ascending=False).transpose()

language_code,eng,en-US,unknown,en-GB,en-CA,spa,fre,ara,ger,ind,...,per,dan,ita,mul,vie,swe,fil,rus,tur,rum
rating,4239947,1179179,337020,111743,47106,25033,10815,10543,3837,2649,...,700,479,307,195,179,129,75,67,67,64


In [40]:
books.loc[(books.language_code.isna() & ~books.title.isna())].head()

,id,book_id,isbn,isbn13,authors,original_publication_year,title,language_code
44,45,4214,770430074,9.780770e+12,Yann Martel,2001.0,Life of Pi,NaN
131,132,3431,1401308589,9.781401e+12,Mitch Albom,2003.0,The Five People You Meet in Heaven,NaN
203,204,5326,1561797464,9.781562e+12,Charles Dickens,1843.0,A Christmas Carol,NaN
236,237,10592,1416524304,9.781417e+12,Stephen King,1974.0,Carrie,NaN
287,288,19691,425172902,9.780425e+12,Tom Clancy,1984.0,"The Hunt for Red October (Jack Ryan Universe, #4)",NaN


use only english

In [41]:
books.loc[(books.original_publication_year.isna() | books.authors.isna() | books.title.isna())].shape[0]

21

In [42]:
from langdetect import detect
books.loc[books['language_code'].isna(), 'language_code'] = [
    detect(title) for title in books.loc[books['language_code'].isna(), 'title']]

In [43]:
#books = books.replace(['eng', 'en-US', 'en-GB', 'en', 'en-CA'], 'en')

In [44]:
books.language_code.unique()

array(['eng', 'en-US', 'en-CA', 'en', 'spa', 'en-GB', 'es', 'fre', 'so',
       'de', 'nl', 'tl', 'af', 'ara', 'id', 'it', 'fr', 'por', 'hu', 'no',
       'ger', 'ro', 'da', 'nor', 'sv', 'jpn', 'vie', 'pt', 'ca', 'et',
       'ind', 'pl', 'pol', 'tr', 'fi', 'cy', 'tur', 'dan', 'fil', 'ita',
       'per', 'sk', 'lt', 'sw', 'cs', 'swe', 'rum', 'mul', 'sl', 'rus'],
      dtype=object)

In [45]:
from langcodes import standardize_tag
books.language_code = books.language_code.apply(standardize_tag)

In [46]:
books.groupby('language_code').aggregate({'id':'count'}).sort_values('id', ascending=False).transpose()

language_code,en,en-US,en-GB,ar,de,en-CA,fr,id,es,da,...,so,lt,sl,sw,hu,sk,ru,cs,mul,vi
id,7233,2070,257,64,63,58,40,30,26,15,...,3,2,1,1,1,1,1,1,1,1


delete missing year, title

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin

class LangDetecter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        from langdetect import detect
        X.loc[X['language_code'].isna(), 'language_code'] = [
            detect(title) for title in X.loc[X['language_code'].isna(), 'title']]
        from langcodes import standardize_tag
        X['language_code'] = X['language_code'].apply(standardize_tag)
        X = X.replace(['eng', 'en-US', 'en-GB', 'en', 'en-CA'], 'en')
        return X

In [48]:
class NonEnglishRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X.loc[
            books['language_code'].isin(['eng', 'en-US', 'en-GB', 'en-CA', 'en']), :
        ].drop('language_code', 1)

class MissingRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X = X.loc[
            (~books.original_publication_year.isna() & 
             ~books.authors.isna() & 
             ~books.title.isna() & 
             ~books.authors.map(lambda x: x == ''))]
        X.loc[:, 'original_publication_year'] = X.loc[:, 'original_publication_year'].astype(int)
        return X.reset_index(drop=True)

# xml parser

In [49]:
import os
from bs4 import BeautifulSoup
import re

In [50]:
def removetags(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', raw_html)
    return cleantext

books_parsed = pd.DataFrame(
    columns=['book_id', 'authors_ids', 'authors_names', 'series', 'title_without_series', 'description']
)

for file in os.listdir('data/books_xml'):
    with open('data/books_xml/' + file, encoding='utf-8') as f:
        b = f.read()
    print(file)
    
    soup = BeautifulSoup(b, 'html.parser')
    
    book_id = int(soup.find('id').contents[0])
    authors_ids = [i.find('id').contents[0] 
                   for i in soup.find('authors').find_all('author') if not i.find('role').contents]
    authors_names = [i.find('name').contents[0] 
                     for i in soup.find('authors').find_all('author') if not i.find('role').contents]
    series = [i.find('title').contents[1].strip('\n').strip(' ') for i in soup.find_all('series')]
    try: title_without_series = soup.find('original_title').contents[0]
    except: title_without_series = np.nan
    try: description = removetags(soup.find('description').contents[0])
    except: description = np.nan
    
    books_parsed = books_parsed.append({
        'book_id':book_id, 'authors_ids':authors_ids, 'authors_names':authors_names, 
        'series':series, 'title_without_series':title_without_series, 'description':description
    }, ignore_index=True)

1.xml
10.xml
10006.xml
1000751.xml
10008056.xml
10009.xml
1001896.xml
100247.xml
10025305.xml
10031.xml
100322.xml
10032672.xml
10033.xml
100365.xml
10037.xml
1003761.xml
100464.xml
10047589.xml
100476.xml
10049436.xml
1005.xml
10054335.xml
10058.xml
10059498.xml
10065.xml
10073.xml
10079321.xml
1008231.xml
10084.xml
10088913.xml
10090.xml
100915.xml
10092.xml
100924.xml
100933.xml
10098912.xml
1009996.xml
10104.xml
10105.xml
10108.xml
10108463.xml
101094.xml
10111087.xml
10112885.xml
1011359.xml
10115.xml
10115852.xml
1011632.xml
101249.xml
101255.xml
10127019.xml
10128428.xml
101299.xml
10131648.xml
1013383.xml
10138607.xml
10140661.xml
10149142.xml
1015311.xml
10155.xml
10161216.xml
10163292.xml
10165727.xml
10169662.xml
10176.xml
101869.xml
101911.xml
10193062.xml
10194157.xml
10194514.xml
10196362.xml
102030.xml
10209997.xml
10210.xml
10211494.xml
10215349.xml
10222.xml
1022866.xml
102324.xml
102327.xml
10235.xml
10238.xml
102439.xml
102458.xml
102459.xml
102461.xml
1025039.xml
10

11597.xml
11600163.xml
116012.xml
11602.xml
116020.xml
11605.xml
116053.xml
116054.xml
11607.xml
11607446.xml
116114.xml
11614718.xml
1162022.xml
11623.xml
116236.xml
1162543.xml
116257.xml
116295.xml
116296.xml
116343.xml
116356.xml
116445.xml
116494.xml
11650.xml
116563.xml
11660590.xml
1166131.xml
11665750.xml
1166599.xml
11666.xml
1168341.xml
11686.xml
11690.xml
11691.xml
1169808.xml
116982.xml
11699349.xml
117.xml
1170158.xml
117047.xml
11709037.xml
1171.xml
11710373.xml
11711.xml
11713.xml
11714997.xml
11720382.xml
11721314.xml
117251.xml
11734.xml
11734251.xml
1173576.xml
11735983.xml
11737271.xml
11737313.xml
11737387.xml
11738128.xml
11741.xml
11742691.xml
11746513.xml
11757975.xml
1175893.xml
11761.xml
11762.xml
11765920.xml
11768.xml
1176884.xml
11777020.xml
1178039.xml
117833.xml
117835.xml
11784281.xml
11797365.xml
11801463.xml
11806716.xml
11808919.xml
11808950.xml
11821514.xml
11824.xml
11827808.xml
11830394.xml
11831.xml
118389.xml
11851522.xml
11857408.xml
11861062.xml

13324841.xml
13325079.xml
133266.xml
13326831.xml
1333030.xml
13330370.xml
13330603.xml
13330761.xml
13330771.xml
13331184.xml
13331199.xml
13333093.xml
13335037.xml
13337715.xml
13337716.xml
133394.xml
133430.xml
133445.xml
13345975.xml
133486.xml
133488.xml
13350.xml
1335019.xml
133518.xml
1335184.xml
133526.xml
13353667.xml
13359067.xml
13360.xml
13360101.xml
13362536.xml
13363270.xml
13366259.xml
133664.xml
13369678.xml
13372690.xml
133729.xml
13374.xml
1337448.xml
13376.xml
133765.xml
133767.xml
13390528.xml
133906.xml
133954.xml
13402447.xml
13406425.xml
13412515.xml
13413589.xml
13414446.xml
13414594.xml
13414599.xml
13414676.xml
13415554.xml
13416236.xml
13417946.xml
13419891.xml
13425393.xml
13425570.xml
13426114.xml
13431827.xml
134353.xml
13435889.xml
134360.xml
13436373.xml
13438524.xml
13440.xml
13448656.xml
13450339.xml
13451.xml
13452375.xml
13453029.xml
13454149.xml
13477676.xml
13477819.xml
1347837.xml
13481275.xml
13486122.xml
13486632.xml
13489517.xml
13489518.xml
13

15790883.xml
15791154.xml
15791403.xml
15793068.xml
15793074.xml
15793165.xml
15793184.xml
15793186.xml
15793231.xml
15795628.xml
15796700.xml
15797397.xml
15797722.xml
15797938.xml
15799151.xml
15799166.xml
157993.xml
15799936.xml
15801381.xml
15801668.xml
15801967.xml
15802944.xml
15803037.xml
15803059.xml
15803141.xml
15803757.xml
15806231.xml
15808287.xml
15808816.xml
15810910.xml
15811495.xml
15811496.xml
15811520.xml
15811545.xml
15811568.xml
158119.xml
15812814.xml
15813668.xml
158141.xml
15814872.xml
15815333.xml
15815364.xml
15818107.xml
15818164.xml
15818278.xml
15818333.xml
15818362.xml
15819028.xml
15820492.xml
15820748.xml
15823426.xml
15823461.xml
15823480.xml
15824358.xml
15827066.xml
15828714.xml
1582996.xml
15832316.xml
15832932.xml
15833693.xml
15835685.xml
15836.xml
15839976.xml
15839984.xml
15842230.xml
15842441.xml
15843166.xml
158446.xml
15849910.xml
158516.xml
15851746.xml
15852.xml
15852479.xml
15852756.xml
15858.xml
15858248.xml
15860.xml
15861491.xml
1587362.x

1753714.xml
1754206.xml
175510.xml
175516.xml
17557143.xml
17557913.xml
17558817.xml
17560674.xml
17562900.xml
17563080.xml
17563539.xml
17565845.xml
17567.xml
1756703.xml
175675.xml
17568801.xml
17569860.xml
17570105.xml
17570538.xml
17571466.xml
17571564.xml
17572903.xml
17573684.xml
175983.xml
17607848.xml
17608111.xml
17617277.xml
17623975.xml
176326.xml
176327.xml
17645.xml
17650.xml
17660.xml
17660462.xml
17661831.xml
17667452.xml
17667561.xml
17671913.xml
17673271.xml
17673307.xml
17674991.xml
17675031.xml
17675462.xml
1767779.xml
176788.xml
17683.xml
176839.xml
1768603.xml
17688.xml
17689.xml
176892.xml
17690.xml
17704902.xml
17704903.xml
17707603.xml
17707605.xml
17707654.xml
17716.xml
17717.xml
17722973.xml
17726978.xml
17727276.xml
17728.xml
1772910.xml
17729712.xml
17733861.xml
17733898.xml
1774836.xml
177515.xml
1776251.xml
17768.xml
17780.xml
17781.xml
17788401.xml
17792606.xml
17794657.xml
17797253.xml
17797381.xml
17798233.xml
17798287.xml
17799.xml
1779929.xml
17800.xm

...

43814.xml
43818.xml
43841.xml
438492.xml
43877.xml
43884.xml
43886.xml
43887.xml
43889.xml
43892.xml
43893.xml
43908.xml
43915.xml
43916.xml
43917.xml
439173.xml
43924.xml
43926.xml
43927.xml
439275.xml
43928.xml
439288.xml
43929.xml
43930.xml
43931.xml
43932.xml
43933.xml
439363.xml
43944.xml
43945.xml
4395.xml
43992.xml
43993.xml
44001.xml
4406.xml
4407.xml
440960.xml
44133.xml
4415692.xml
441618.xml
44170.xml
44184.xml
44186.xml
44255.xml
443540.xml
44360.xml
44361.xml
44399.xml
444304.xml
444347.xml
444381.xml
4447622.xml
4452257.xml
44543.xml
44559.xml
44597.xml
446.xml
44644.xml
44652.xml
44659.xml
44660.xml
44687.xml
44688.xml
447.xml
4473.xml
4477.xml
44796.xml
4485966.xml
448836.xml
4488657.xml
448873.xml
449.xml
44905.xml
4494917.xml
4497978.xml
4499214.xml
4499669.xml
45020.xml
4502507.xml
4502877.xml
45032.xml
4505161.xml
450573.xml
4507.xml
450836.xml
45100.xml
45101.xml
45102.xml
45106.xml
45107.xml
45108.xml
45109.xml
45110.xml
45112.xml
4516.xml
45162.xml
45169.xml
4518

567678.xml
567704.xml
567795.xml
568099.xml
568236.xml
56842.xml
568454.xml
568476.xml
568592.xml
568645.xml
56899.xml
56912.xml
56913.xml
56919.xml
5693.xml
569346.xml
5695.xml
569564.xml
5697.xml
570172.xml
571508.xml
571555.xml
5720.xml
573201.xml
5739373.xml
5750628.xml
5752.xml
5755.xml
5759.xml
57605.xml
5764902.xml
576570.xml
576666.xml
5771014.xml
57727.xml
577283.xml
57736.xml
577542.xml
5776788.xml
57854.xml
5787.xml
57891.xml
57913.xml
579143.xml
57951.xml
579696.xml
5797.xml
58027.xml
58033.xml
5805.xml
58077.xml
5809.xml
58098.xml
581125.xml
581409.xml
581552.xml
5821978.xml
582533.xml
5826.xml
58314.xml
58345.xml
583924.xml
5845.xml
584637.xml
5849.xml
58547.xml
5861926.xml
586472.xml
586516.xml
58696.xml
5872.xml
58728.xml
587582.xml
587638.xml
587666.xml
588138.xml
58822.xml
588234.xml
588369.xml
5883777.xml
5886881.xml
588747.xml
58889.xml
58890.xml
58893.xml
58894.xml
5890.xml
58922.xml
58962.xml
5899779.xml
589979.xml
590552.xml
5907.xml
59091.xml
59142.xml
59145.xml

6697.xml
6697305.xml
669780.xml
67.xml
67035.xml
6708.xml
6709.xml
6713071.xml
671724.xml
6718608.xml
6723348.xml
67238.xml
672493.xml
6726595.xml
6728738.xml
672875.xml
672948.xml
6732019.xml
673252.xml
6735082.xml
673657.xml
6736971.xml
674220.xml
6746018.xml
6748.xml
6749.xml
6750.xml
6751.xml
67513.xml
6751356.xml
6752378.xml
6753.xml
675614.xml
6758331.xml
6758423.xml
6759.xml
675905.xml
676.xml
6763961.xml
676555.xml
67657.xml
6765740.xml
676737.xml
676924.xml
6777616.xml
6780439.xml
6781956.xml
67863.xml
6791212.xml
67920.xml
6792458.xml
67931.xml
6801582.xml
6801614.xml
6801652.xml
68099.xml
68143.xml
6818019.xml
6819.xml
6820.xml
68210.xml
682153.xml
682745.xml
682751.xml
682761.xml
682793.xml
682804.xml
6828896.xml
682920.xml
6834410.xml
6837103.xml
68378.xml
68380.xml
6839093.xml
68396.xml
68418.xml
68427.xml
68428.xml
68429.xml
68452.xml
68453.xml
68455.xml
68458.xml
68483.xml
68487.xml
68488.xml
68494.xml
68496.xml
68497.xml
68520.xml
68521.xml
68524.xml
68526.xml
68527.xm

7821828.xml
7822895.xml
7823038.xml
7823678.xml
7824322.xml
7824768.xml
7824997.xml
78250.xml
7825557.xml
782580.xml
7826116.xml
7827221.xml
782854.xml
7828781.xml
7828882.xml
7830126.xml
7831145.xml
783127.xml
7831742.xml
783291.xml
7840156.xml
78408.xml
78411.xml
7841446.xml
7841677.xml
78418.xml
784222.xml
7842288.xml
78427.xml
78433.xml
7849034.xml
784911.xml
7849341.xml
78508.xml
785092.xml
7853137.xml
7853757.xml
785453.xml
785454.xml
7856305.xml
7857195.xml
786256.xml
7862725.xml
78629.xml
7863274.xml
7863423.xml
7864437.xml
78674.xml
7869.xml
7870457.xml
78728.xml
787660.xml
7879278.xml
7882.xml
78876.xml
789344.xml
7894870.xml
78950.xml
789559.xml
78960.xml
7896345.xml
7896527.xml
78970.xml
78971.xml
78972.xml
7897478.xml
78976.xml
7897620.xml
78982.xml
78983.xml
78985.xml
78987.xml
78989.xml
79006.xml
79016.xml
790171.xml
7904429.xml
7904453.xml
7905092.xml
7905977.xml
7906105.xml
7907680.xml
7907782.xml
7907803.xml
7908638.xml
7909.xml
7911.xml
7912007.xml
79121.xml
7913305.

914211.xml
91439.xml
91440.xml
91474.xml
91475.xml
91476.xml
91477.xml
91478.xml
91479.xml
9151181.xml
91571.xml
91582.xml
9160695.xml
9165406.xml
91661.xml
9166559.xml
9166877.xml
91714.xml
91760.xml
91767.xml
9182478.xml
918789.xml
91953.xml
91981.xml
91989.xml
920143.xml
92057.xml
920607.xml
920649.xml
920780.xml
92121.xml
92143.xml
92147.xml
92160.xml
92163.xml
921651.xml
9219901.xml
92227.xml
92250.xml
92254.xml
922991.xml
92303.xml
92307.xml
92308.xml
92364.xml
92367.xml
92371.xml
92373.xml
92375.xml
923950.xml
92508.xml
92517.xml
925367.xml
9260507.xml
9265453.xml
9265891.xml
9266769.xml
9266776.xml
9266793.xml
9266810.xml
9268487.xml
926908.xml
9275658.xml
92780.xml
9279177.xml
9279734.xml
927979.xml
9282.xml
92845.xml
92846.xml
9284655.xml
92855.xml
92871.xml
92918.xml
9293020.xml
92956.xml
9296438.xml
9297774.xml
929782.xml
9298.xml
93.xml
930.xml
9300.xml
93000.xml
93007.xml
9300768.xml
9301.xml
9302.xml
9303.xml
9305362.xml
930612.xml
9307674.xml
93101.xml
93157.xml
93261.x

In [51]:
needed_vars = ['id', 'book_id', 'isbn', 'isbn13', 'authors', 
               'original_publication_year', 'title', 'language_code']
books_join = pd.merge(
    pd.read_csv('data/books.csv')[needed_vars], 
    books_parsed, 
    on='book_id'
)
books_join.head()

,id,book_id,isbn,isbn13,authors,original_publication_year,title,language_code,authors_ids,authors_names,series,title_without_series,description
0,1,2767052,439023483,9.780439e+12,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)",eng,[153394],[Suzanne Collins],[The Hunger Games],The Hunger Games,Winning will make you famous. Losing means c...
1,2,3,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,eng,[1077326],[J.K. Rowling],[Harry Potter],Harry Potter and the Philosopher's Stone,Harry Potter's life is miserable. His parents ...
2,3,41865,316015849,9.780316e+12,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)",en-US,[941441],[Stephenie Meyer],[Twilight],Twilight,About three things I was absolutely positive....
3,4,2657,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,eng,[1825],[Harper Lee],[To Kill a Mockingbird],To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...
4,5,4671,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,[3190],[F. Scott Fitzgerald],[],The Great Gatsby,"THE GREAT GATSBY , F. Scott Fitzgerald’s thir..."


In [52]:
class AuthorsDeLister(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        def listtostring(l):
            return ", ".join(l)
        X['authors'] = X['authors_names'].map(listtostring)
        return X.drop('authors_names', 1)
    
class SeriesTaker(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        def firstoflist(l):
            return l[0] if len(l)>0 else ''
        X['series'] = X['series'].map(firstoflist)
        return X

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer
books_authors = MultiLabelBinarizer(sparse_output=True).fit_transform(books_join.loc[:, 'authors_ids'])
#sp.save_npz('data/authors.npz', books_authors)
books_authors

<10000x4293 sparse matrix of type '<class 'numpy.int32'>'
	with 10689 stored elements in Compressed Sparse Row format>

In [54]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('authors', AuthorsDeLister()), 
                 ('series', SeriesTaker()),
                 ('lang', LangDetecter())
                 #('english', NonEnglishRemover()), 
                 #('missing', MissingRemover()),
                ])
books_transform = pipe.fit_transform(books_join)
books_transform.head()

,id,book_id,isbn,isbn13,authors,original_publication_year,title,language_code,authors_ids,series,title_without_series,description
0,1,2767052,439023483,9.780439e+12,Suzanne Collins,2008.0,"The Hunger Games (The Hunger Games, #1)",en,[153394],The Hunger Games,The Hunger Games,Winning will make you famous. Losing means c...
1,2,3,439554934,9.780440e+12,J.K. Rowling,1997.0,Harry Potter and the Sorcerer's Stone (Harry P...,en,[1077326],Harry Potter,Harry Potter and the Philosopher's Stone,Harry Potter's life is miserable. His parents ...
2,3,41865,316015849,9.780316e+12,Stephenie Meyer,2005.0,"Twilight (Twilight, #1)",en,[941441],Twilight,Twilight,About three things I was absolutely positive....
3,4,2657,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,en,[1825],To Kill a Mockingbird,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...
4,5,4671,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,en,[3190],,The Great Gatsby,"THE GREAT GATSBY , F. Scott Fitzgerald’s thir..."


In [55]:
books_transform.drop('authors_ids', 1).to_csv('data/books_transform.csv', index=False)

In [57]:
books_lang = sp.csr_matrix(
    pd.get_dummies(books_transform['language_code'], columns=['language_code']).values
)
#sp.save_npz('data/languages.npz', books_lang)
books_lang

<10000x34 sparse matrix of type '<class 'numpy.uint8'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [58]:
books_authors = MultiLabelBinarizer(sparse_output=True).fit_transform(
    books_transform.loc[:, 'authors_ids']
)

# tags

In [59]:
tags = pd.read_csv('data/tags.csv')
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


In [60]:
book_tags = pd.read_csv('data/book_tags.csv')
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [61]:
relative_tags = pd.merge(
    book_tags,
    book_tags.groupby('goodreads_book_id').aggregate({'count':'sum'}).reset_index(), 
    on='goodreads_book_id'
)
relative_tags['count_percent'] = relative_tags['count_x'] / relative_tags['count_y']
relative_tags = relative_tags.drop(['count_x', 'count_y'], 1)
relative_tags.head()

,goodreads_book_id,tag_id,count_percent
0,1,30574,0.466542
1,1,11305,0.103420
2,1,11557,0.095071
3,1,8717,0.036128
4,1,33114,0.035377


In [62]:
relative_tags_cut = relative_tags.loc[relative_tags['count_percent'] > 0.01, :]
relative_tags_cut.head()

,goodreads_book_id,tag_id,count_percent
0,1,30574,0.466542
1,1,11305,0.103420
2,1,11557,0.095071
3,1,8717,0.036128
4,1,33114,0.035377


In [63]:
relative_tags_cut = pd.merge(
    relative_tags_cut, 
    books[['book_id', 'id']], 
    left_on='goodreads_book_id', 
    right_on='book_id').drop(['goodreads_book_id', 'book_id'], 1).sort_values('id').reset_index(drop=True)
relative_tags_cut.head()

,tag_id,count_percent,id
0,30574,0.039354,1
1,26138,0.011621,1
2,11590,0.014588,1
3,22743,0.017058,1
4,27199,0.017719,1


In [64]:
#relative_tags_cut = relative_tags_cut.loc[relative_tags_cut['id'].isin(books_transform['id']), :]

In [65]:
relative_tags_cut.tag_id.nunique()

2498

In [66]:
revert = dict(zip(
    relative_tags_cut['tag_id'].unique(), 
    np.arange(len(relative_tags_cut.tag_id.unique())),
))

In [67]:
relative_tags_cut['tag_id'] = relative_tags_cut['tag_id'].replace(revert)

In [68]:
relative_tags_cut

,tag_id,count_percent,id
0,0,0.039354,1
1,1,0.011621,1
2,2,0.014588,1
3,3,0.017058,1
4,4,0.017719,1
...,...,...,...
99019,0,0.857182,9999
99020,66,0.013553,10000
99021,16,0.016090,10000
99022,63,0.048023,10000


In [69]:
books_genres = sp.csr_matrix(
    (relative_tags_cut['count_percent'], 
     (relative_tags_cut['id']-1, relative_tags_cut['tag_id'])
    )
)
#sp.save_npz('data/genres.npz', genres)
books_genres

<10000x2498 sparse matrix of type '<class 'numpy.float64'>'
	with 99024 stored elements in Compressed Sparse Row format>

In [72]:
from sklearn.preprocessing import normalize
books_year = books['original_publication_year'].fillna(books['original_publication_year'].mean())
books_year = normalize(
    books_year.values.reshape(-1, 1),
    norm='l1', 
    axis=0
)

In [73]:
features = sp.hstack((
    books_genres,
    books_authors,
    books_lang,
    books_year,
)).tocsr()

item_features = sp.hstack((
    features,
    sp.coo_matrix((features.shape[0], features.shape[0] - features.shape[1])),
))
sp.save_npz('data/item_features.npz', item_features)